# LangChain 核心模块 Agent - Self ask with search

- Google 搜索对接
    - 第三方平台：https://serpapi.com
    - LangChain API 封装：`SerpAPIWrapper`
- LangChain Agent 模块使用
    - Agent 类型：`SELF_ASK_WITH_SEARCH`
    - Agent 实例化：`initialize_agent` 方法
 
https://python.langchain.com/docs/modules/agents/agent_types/

In [1]:
import os

# 更换为自己的 Serp API KEY
os.environ["SERPAPI_API_KEY"] = "1e3c93e0753ac224098370cd71da86150c6609caba6b5aaa307e04b72a5006e1"

In [2]:
from langchain import OpenAI, SerpAPIWrapper
from langchain.agents import initialize_agent, AgentType, Tool

llm = OpenAI(temperature=0)

In [3]:
# 实例化查询工具
search = SerpAPIWrapper()
tools = [
    Tool(
        name="Intermediate Answer",
        func=search.run,
        description="useful for when you need to ask with search",
    )
]

In [4]:
# 实例化 SELF_ASK_WITH_SEARCH Agent
self_ask_with_search = initialize_agent(
    tools, llm, agent=AgentType.SELF_ASK_WITH_SEARCH, verbose=True
)

In [5]:
# 实际运行 Agent，查询问题（正确）
self_ask_with_search.run(
    "成都举办的大运会是第几届大运会？"
)



> Entering new AgentExecutor chain...
 Yes.
Follow up: 什么时候举办的成都大运会？
Intermediate answer: Fri, Jul 28, 2023, 8:00 AM – Tue, Aug 8, 2023
Follow up: 成都大运会是第几届大运会？
Intermediate answer: 2021年夏季世界大学生运动会（英语：2021 Summer World University Games）简称为2021年成都大运会或成都大运会，是第31届夏季世界大学生运动会，原定于2021年于中国四川省成都市举行，后因2019冠状病毒病疫情等原因二次延期，最终改至2023年7月28日至8月8日举行。
So the final answer is: 第31届

> Finished chain.


'第31届'

In [6]:
# 实际运行 Agent，查询问题（有歧义）
self_ask_with_search.run(
    "2023年大运会举办地在哪里？"
)



> Entering new AgentExecutor chain...
 Yes.
Follow up: 什么是2023年大运会？
Intermediate answer: ['The 2023 FISU Summer World University Games, officially known as the 2023 Summer Universiade or the XXXII Summer World University Games, and commonly known as Yekaterinburg 2023 or Ekaterinburg 2023, ...', '2023 Summer World University Games (2023年夏季世界大学生运动会) type: Aug 8, 2023 – Aug 19, 2023.', '2023 Summer World University Games (2023年夏季世界大学生运动会) main_tab_text: Overview.', '2023 Summer World University Games (2023年夏季世界大学生运动会) kgmid: /g/11hzkdc1v8.', '2023 Summer World University Games (2023年夏季世界大学生运动会) dates: Aug 8, 2023 – Aug 19, 2023.', '2023 Summer World University Games (2023年夏季世界大学生运动会) location: Yekaterinburg.', '2023 Summer World University Games (2023年夏季世界大学生运动会) main_venue: Ekaterinburg Arena (opening/closing ceremonies).', '2023 Summer World University Games (2023年夏季世界大学生运动会) motto: For You.', '2023 Summer World University Games (2023年夏季世界大学生运动会) opening: 8 August (cancelled).', '202

'成都'

In [7]:
# Reason-only 答案（错误）
print(llm("成都举办的大运会是第几届大运会？"))



成都举办的是第十一届大运会。


In [8]:
# Reason-only 答案（错误）
print(llm("2023年大运会举办地在哪里？"))



2023年大运会将在日本东京举办。


#### 使用 GPT-4 作为大语言模型实现更优的 ReAct 范式

In [9]:
from langchain.chat_models import ChatOpenAI

chat_model = ChatOpenAI(model="gpt-4-1106-preview", temperature=0)

In [10]:
self_ask_with_search_chat = initialize_agent(
    tools, chat_model, agent=AgentType.SELF_ASK_WITH_SEARCH, verbose=True
)

In [11]:
# GPT-4 based ReAct 答案（正确）
self_ask_with_search_chat.run(
    "2023年大运会举办地在哪里？"
)



> Entering new AgentExecutor chain...
Yes.
Follow up: What is the English translation of "2023年大运会"?
Intermediate answer: No good search result found
The phrase "2023年大运会" translates to "2023 Universiade" in English. The Universiade is an international multi-sport event, organized for university athletes by the International University Sports Federation (FISU).

Follow up: Where is the 2023 Universiade being held?
Intermediate answer: Lake Placid 2023 FISU World University Games Winter, 12-22 January. Welcome back to legendary Lake Placid, where many Universiade and Olympic Legends were born.
So the final answer is: Lake Placid, New York, United States

> Finished chain.


'Lake Placid, New York, United States'